In [1]:
import numpy as np
import time

In [47]:
M = np.array([[1, 2, 0], [1, 3, 1], [-2, 0, 1]], np.double)
b = np.array([3, 5, -1], np.double).reshape((3, 1))
dim = b.shape[0]

# Exercise 1

In [48]:
def triangular_lower_solve(L, b, dim):
    # Solve Ly = b
    y = np.zeros((dim, 1))
    for i in range(0, dim): # Iterate over each L line
        y[i] = b[i]
        for j in range(0, i): # Iterate over columns until i
            y[i] -= L[i][j]*y[j]
        y[i] /= L[i][i]
    
    return y

L_test = np.array([[1, 0, 0], [2, 1, 0], [3, 2, 1]], np.double)
b_test = np.array([-1, 0, 6], np.double).reshape((3, 1))

triangular_lower_solve(L_test, b_test, 3)

array([[-1.],
       [ 2.],
       [ 5.]])

In [49]:
def triangular_upper_solve(U, y, dim):
    # Solve Ux = y
    x = np.zeros((dim, 1))
    for i in range(dim-1, -1, -1): # Iterate over each U line, reversed
        x[i] = y[i]
        for j in range(dim-1, i, -1): # Iterate over columns until 0, from right to left
            x[i] -= U[i][j]*x[j]
        x[i] /= U[i][i]
    
    return x

U_test = np.array([[2, 1, 2], [0, 1, -1], [0, 0, -5]], np.double)
y_test = np.array([-1, 2, 5], np.double).reshape((3, 1))

triangular_upper_solve(U_test, y_test, 3)

array([[ 0.],
       [ 1.],
       [-1.]])

# Exercise 2

In [2]:
def lu_factorization(M, dim):
    L = np.identity(dim)
    U = np.zeros((dim,dim))
    
    U[0] = M[0] # Iterate first line
    
    for it in range(0, dim-1):
        
        for line in range(it+1, dim): # Iterate it'th column
            sum = 0
            for it2 in range(0, it+1):
                sum += L[line][it2]*U[it2][it]
            L[line][it] = (M[line][it] - sum)/U[it][it]
        
        for col in range(it+1, dim): # Iterate (it+1)'th line
            sum = 0
            for it2 in range(0, it+2):
                sum += L[it+1][it2]*U[it2][col]
            U[it+1][col] = M[it+1][col] - sum
        
    return L, U

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
L_test, U_test = lu_factorization(M_test, 3)

print(L_test)
print(U_test)

np.allclose(L_test @ U_test, M_test)

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 2. 1.]]
[[ 2.  1.  2.]
 [ 0.  1. -1.]
 [ 0.  0. -5.]]


True

In [74]:
L, U = lu_factorization(M, dim)

# Solve Ly = b
y = triangular_lower_solve(L, b, dim)

# Solve Ux = y
x = triangular_upper_solve(U, y, dim)

x2 = np.linalg.solve(M, b)

print('x \n', x)
print()
print('x2\n', x2)
np.allclose(x, x2)

x 
 [[1.]
 [1.]
 [1.]]

x2
 [[1.]
 [1.]
 [1.]]


True

# Exercise 3

In [52]:
def gauss_elimination(M, b, dim):
    
    U = M.copy()
    y = b.copy()
    
    for col in range(0, dim-1):
        col_value = U[col][col]
        for line in range(col+1, dim):
            multiplier = -U[line][col]/col_value
            U[line] = U[col]*multiplier + U[line]
            y[line] = y[col]*multiplier + y[line]
    
    return triangular_upper_solve(U, y, dim)

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
b_test = np.array([-1, 0, 6], np.double).reshape((3, 1))

gauss_elimination(M_test, b_test, 3)

array([[ 0.],
       [ 1.],
       [-1.]])

# Exercise 4

In [53]:
def gauss_elimination_with_pivot(M, b, dim):
    
    U = M.copy()
    y = b.copy()
    
    for col in range(0, dim-1):
        
        current_pivot = np.absolute(U[col:,col]).argmax(axis=0)+col
        U[[col, current_pivot]] = U[[current_pivot, col]]
        y[[col, current_pivot]] = y[[current_pivot, col]]
        
        col_value = U[col][col]
        for line in range(col+1, dim):
            multiplier = -U[line][col]/col_value
            U[line] = U[col]*multiplier + U[line]
            y[line] = y[col]*multiplier + y[line]
            
    return triangular_upper_solve(U, y, dim)

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
b_test = np.array([-1, 0, 6], np.double).reshape((3, 1))

gauss_elimination_with_pivot(M_test, b_test, 3)

array([[ 0.],
       [ 1.],
       [-1.]])

# Exercise 5

In [27]:
def lu_factorization_with_gauss_pivot(M, dim):
    U = M.copy()
    L = np.zeros((dim,dim), dtype=np.double)
    P = np.identity(dim, dtype=np.double)
    
    for col in range(0, dim-1):
        
        current_pivot = np.absolute(U[col:,col]).argmax(axis=0)+col
        U[[col, current_pivot]] = U[[current_pivot, col]]
        L[[col, current_pivot]] = L[[current_pivot, col]]
        P[[col, current_pivot]] = P[[current_pivot, col]]
        
        col_value = U[col][col]
        for line in range(col+1, dim):
            multiplier = -U[line][col]/col_value
            L[line][col] = -multiplier
            U[line] = U[col]*multiplier + U[line]
    L += np.identity(dim, dtype=np.double)        
    return L, U, P

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
L_test, U_test, P_test = lu_factorization_with_gauss_pivot(M_test, 3)

print(L_test)
print(U_test)
print(P_test)

np.allclose(P_test.T @ L_test @ U_test, M_test)

[[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [0.66666667 0.5        1.        ]]
[[ 6.          5.         -1.        ]
 [ 0.         -0.66666667  2.33333333]
 [ 0.          0.          2.5       ]]
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


True

# Exercise 6

In [75]:
def inverse_matrix(M, dim):
    M_inv = np.zeros((dim,dim), dtype=np.double)
    I = np.identity(dim, dtype=np.double)
    
    for col in range(0, dim):
        M_inv[:, col] = gauss_elimination_with_pivot(M, I[:,col], dim).reshape((3,))
    
    return M_inv

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
M_test_inv = inverse_matrix(M_test, 3)

print(M_test_inv)
np.allclose(M_test @ M_test_inv, np.identity(3, dtype=np.double))

[[ 1.8 -1.1  0.3]
 [-2.2  1.4 -0.2]
 [-0.2  0.4 -0.2]]


True

# Exercise 7

In [3]:
def lu_factorization_for_band_matrix(M, p, dim):
    L = np.identity(dim)
    U = np.zeros((dim,dim))
    
    U[0] = M[0] # Iterate first line
    
    for it in range(0, dim-1):
        
        for line in range(it+1, min(it+1+p,dim)): # Iterate it'th column
            sum = 0
            for it2 in range(max(0, line-p), it+1):
                sum += L[line][it2]*U[it2][it]
            L[line][it] = (M[line][it] - sum)/U[it][it]
        
        for col in range(it+1, min(it+2+p, dim)): # Iterate (it+1)'th line
            sum = 0
            for it2 in range(max(0, col-p), it+2):
                sum += L[it+1][it2]*U[it2][col]
            U[it+1][col] = M[it+1][col] - sum
        
    return L, U

M_test = np.array([[1, -2, -3, 0, 0], [2, 4, 2, 4, 0], [1, 1, 5, -9, 8], [0, -5, 7, 6, 2], [0, 0, 4, 2, 7]], np.double)
L_test, U_test = lu_factorization_for_band_matrix(M_test, 2, 5)

print(L_test)
print(U_test)

np.allclose(L_test @ U_test, M_test)

[[ 1.          0.          0.          0.          0.        ]
 [ 2.          1.          0.          0.          0.        ]
 [ 1.          0.375       1.          0.          0.        ]
 [ 0.         -0.625       2.4         1.          0.        ]
 [ 0.          0.          0.8         0.30860534  1.        ]]
[[  1.          -2.          -3.           0.           0.        ]
 [  0.           8.           8.           4.           0.        ]
 [  0.           0.           5.         -10.5          8.        ]
 [  0.           0.           0.          33.7        -17.2       ]
 [  0.           0.           0.           0.           5.90801187]]


True

In [29]:
# LU vs LU for band
dim = 500
p = 2

M_test = np.zeros((dim,dim))
for i in range(dim):
    for j in range(max(0,i-p), min(i+p+1, dim)):
        M_test[i, j] = np.random.normal()  
        
start_time = time.time()
L_test, U_test, P_test = lu_factorization_with_gauss_pivot(M_test, dim)
end_time = time.time()
print(end_time - start_time)

start_time = time.time()
L_test_band, U_test_band = lu_factorization_for_band_matrix(M_test, p, dim)
end_time = time.time()
print(end_time - start_time)

print(np.allclose(P_test.T @ L_test @ U_test, L_test_band @ U_test_band))
print(np.allclose(L_test @ U_test, M_test))

0.5054197311401367
0.005908012390136719
True
False
